In [ ]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_ext.tools.mcp import  StreamableHttpMcpToolAdapter, StreamableHttpServerParams
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from azure.identity import AzureCliCredential, get_bearer_token_provider
import asyncio
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_ext.tools.mcp import  StreamableHttpMcpToolAdapter, StreamableHttpServerParams
async def main() -> None:
    # Create server params for the remote MCP service
    server_params = StreamableHttpServerParams(
        url="http://127.0.0.1:8000/mcp",   # update it with deployed url if available
        headers={"Authorization": "Bearer your-api-key", "Content-Type": "application/json"},
        timeout=60,
        
    )

    credential = AzureCliCredential() # use DefaultAzureCredential if cli not available
    token_provider =  get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")

    # Get the translation tool from the server
    video_agent_tool_adapter = await StreamableHttpMcpToolAdapter.from_server_params(server_params, "video_agent_tool")
    kb_tool_adapter = await StreamableHttpMcpToolAdapter.from_server_params(server_params,"kb_tool")

    # Create an agent that can use the translation tool
    model_client = AzureOpenAIChatCompletionClient(
        azure_endpoint="https://<your_resource_name>.openai.azure.com/",
        azure_ad_token_provider=token_provider,
        api_version="2025-01-01-preview",
        model="gpt-4o"
        # api_key="api-key"  # if authorization is not via managed identity
    )

    agent = AssistantAgent(
        name="video_agent",
        model_client=model_client,
        tools=[video_agent_tool_adapter, kb_tool_adapter],
        system_message="""You are a Video Agent whose role is to perform relevant tool calling and return answer according to user query.""",
    )

    # Let the agent translate some text
    await Console(
        agent.run_stream(
            task="""
            query: <query>,
            index: <index-name>,
            additional info: 
            """, cancellation_token=CancellationToken())
    )


if __name__ == "__main__":
    asyncio.run(main())